# Data Exploration
* Data from: https://open.fda.gov/apis/drug/event/download/
* API: https://api.fda.gov/drug/event.json

In [ ]:
# import libraries
import pandas as pd

### Load all data

In [ ]:
filenames = [f'./data/data_2018_q{i}.csv' for i in [1, 2, 3, 4]]
df_list = []
for f in filenames:
    print(f'reading file {f}')
    df_list.append(pd.read_csv(f))
df_original = pd.concat(df_list, ignore_index=True, sort=True)
del df_list
df_original.head(20)